In [1]:
# deafaultdict выбран из-за своей функциональности (ссылка в README.md)
from collections import defaultdict

In [5]:
def solve():

    ## Чтение данных из файла

    # Чтение данных из файла logs.csv
    with open('logs.csv', 'r') as f:
        # Читает построчно файл, создает список с элеменатами - прочитанными строками
        lines = f.readlines()
    # При обращении к несуществующему ключу, автоматически будет создаваться пустое множество set()
    email_to_phones = defaultdict(set)
    phone_to_emails = defaultdict(set)

    ## Создание множеств для телефонов и почт пользователей

    # Пробег по каждой строчке
    for line in lines:
        # .strip() удаляет пробел и переход на новую строку \n в начале и конце строки, 
        # Далее строка по запятой разбивается на список
        parts = line.strip().split(',')
        # Структура фиксированная, поэтому первая часть - email, вторая - телефон
        email = parts[0].strip()
        # Но надо проверить, что номер не пустой
        phone = parts[1].strip() if len(parts) > 1 else ''
        # Проверка на непустые значения email и phone: None, '', пробел
        if email and phone:
            # К соответсвующему ключу добавляется телефон
            email_to_phones[email].add(phone)
            # Email
            phone_to_emails[phone].add(email)
    # Новые множества для email и телефонов
    visited_emails = set()
    visited_phones = set()
    # Для хранения итоговых групп пользвателей
    groups = []
    # Пробег по email в словаре соответсвия email -> телефон
    for email in email_to_phones:
        # Если email еще не фиксировался в посещениях
        if email not in visited_emails:
            # Создание множества для email и телефона
            group_emails = set()
            group_phones = set()
            # В очередь присваивается email
            queue = [email]
            # Пока очередь непустая
            while queue:
                # Извлекает текущее значение в очереди .pop() - последнее
                current = queue.pop()
                # Если текущее значение в ключах словаря email -> телефон
                if current in  email_to_phones:
                    # То это email, добавляет соответсвующий ему телефон
                    for phone in email_to_phones[current]:
                        # Если телефон еще не группировался
                        if phone not in group_phones:
                            # То в множество с телефонами добавляется phone
                            group_phones.add(phone)
                            # И в очередь
                            queue.append(phone)
                    # В множество email-ов добавляется текущий
                    group_emails.add(current)
                    # В множество email-ов, посещавших сайт, добавляется текущий
                    visited_emails.add(current)
                else:
                    # Это телефон, нужно добавить email
                    for email in phone_to_emails[current]:
                        # Если email еще не группировался
                        if email not in group_emails:
                            # То в множество с почтами добавляется email
                            group_emails.add(email)
                            # И в очередь
                            queue.append(email)
                    # В множество телефонов добавляется текущий
                    group_phones.add(current)
                    # В множество телефонов, посетивших сайт, добавляется текущий
                    visited_phones.add(current)
            # В итоговую группу пользователей добавляется мноежство email и множество телефонов
            groups.append((group_emails, group_phones))
    
    ## Создание соответствий email/телефона

    # Создаем словари для связи email/телефона с номером группы
    email_group = {}  # {email: номер_группы}
    phone_group = {}  # {телефон: номер_группы}
    # Нумерация групп, соответсвующих сету email и телефонов
    for group_num, (emails_in_group, phones_in_group) in enumerate(groups):
        # Связь email этой группы с ее номером
        for email in emails_in_group:
            email_group[email] = group_num
        # Связь телефонов этой группы с ее номером
        for phone in phones_in_group:
            phones_in_group[phone] = group_num
    
    ## Подсчет количества транзакций для каждой группы

    # Создание списков с нулями для каждой группы
    transactions_count = [0]*len(groups)
    # Разбиение строки на email и телефон
    for line in lines:
        parts = line.strip().split(',')
        email = parts[0].strip()
        phone = parts[1].strip() if len(parts) > 1 else ''
    # Изначально присывается пустое занчение ноемру группы
    current_group = None
    # Сначала проверяется email: значение непустое и email есть в списке групп
    if email and email in email_group:
        current_group = email_group[email]
    # Если email не найден -> проверка по телефону
    elif phone and phone in phone_group:
        current_group = phone_group[phone]
    # Группа найдена -> увеличивает счетчик
    if current_group is not None:
        transactions_count[current_group] += 1
    # Нахождение максимального количества транзакций
    if not transactions_count: # Если групп нет
        print(0)
    else:
        print(max(transactions_count))

In [6]:
solve()

TypeError: 'set' object does not support item assignment